In [1]:
import cv2
import torch

image = cv2.imread('514508.png')
image = cv2.resize(image, (224,224))
image = torch.tensor(image, dtype=torch.float)
image = image.unsqueeze(0)
image = torch.cat((image, image),  dim = 0)
image.shape

torch.Size([2, 224, 224, 3])

In [2]:
from model import Layer

input_layer = Layer.Input_layer()
encoder_layer = Layer.Encoder_layer()
decoder_layer = Layer.Decoder_layer()
cra = Layer.CRA()

In [3]:
x = input_layer(image)
x = encoder_layer(x)
y = encoder_layer(x)
x = x[:,1:,:]
y = y[:,1:,:]
# x = x.reshape(2, 14, 14, 768)
# x = x.permute(0, 3, 1, 2)
tmp = cra(x, y)
tmp.shape

torch.Size([2, 768, 196])

In [4]:
tmp.shape

torch.Size([2, 768, 196])

In [5]:
from model.ViT_CRA import ViT_CRA

model = ViT_CRA()
x, y = model(image)

In [7]:
y.shape

torch.Size([2, 1, 28, 28])

In [8]:
import torch.nn as nn
import torch
import numpy as np

def Contrastive_convolution_kernel(map, index):
    # Define a custom 2D convolution kernel (3x3 kernel)
    custom_kernel = torch.tensor([[0, 0, 0],
                                [0, 1, 0],
                                [0, 0, 0]], dtype=torch.float32)
    x = index // 3
    y = index % 3
    custom_kernel[x][y] = -1
    map = map.unsqueeze(0).unsqueeze(0)
    # Reshape the kernel to match PyTorch's expected shape
    # (out_channels, in_channels, kernel_height, kernel_width)
    custom_kernel = custom_kernel.view(1, 1, 3, 3)

    # Create a custom convolutional layer using the custom kernel
    custom_conv_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False, padding = 1)
    custom_conv_layer.weight.data = custom_kernel

    # Apply the custom convolution layer to the input tensor
    output_tensor = custom_conv_layer(map)
    return output_tensor[0,0,:,:]

map = np.ones((28, 28))
map = torch.tensor(map, dtype= torch.float32)
Contrastive_convolution_kernel(map, 0).shape

torch.Size([28, 28])